In [1]:
import numpy as np
import tensorflow as tf
import gym
import mujoco_py

np.random.seed(2)
tf.set_random_seed(2)  # reproducible

# Superparameters
OUTPUT_GRAPH = False
MAX_EPISODE = 50000
DISPLAY_REWARD_THRESHOLD = -10  # renders environment if total episode reward is greater then this threshold
MAX_EP_STEPS = 100 # maximum time step in one episode
RENDER = False # rendering wastes time
GAMMA = 0.9     # reward discount in TD error
ALPHA = 3
LR_A = 0.0001    # learning rate for actor
LR_C = 0.0001    # learning rate for critic
# env = gym.make('MountainCarContinuous-v0')
env = gym.make('Reacher-v1')
env.seed(1)  # reproducible

N_F = env.observation_space.shape[0]
# N_A = 3
# action_map = np.linspace(env.action_space.low[0],env.action_space.high[0],num=N_A)

# Reacher
action_res = [11, 11]
N_A = action_res[0]*action_res[1]
action_map = np.zeros([np.prod(action_res), 2])
u = np.linspace(env.action_space.low[0], env.action_space.high[0], num=action_res[0])
v = np.linspace(env.action_space.low[1], env.action_space.high[1], num=action_res[1])
for i in range(action_res[0]):
    for j in range(action_res[1]):
        s = action_res[1] * i + j
        action_map[s, :] = [u[i], v[j]]
'''
action_res = [6, 6, 6]
N_A = 6*6*6
action_map = np.zeros([np.prod(action_res), 3])
u = np.linspace(env.action_space.low[0], env.action_space.high[0], num=action_res[0])
v = np.linspace(env.action_space.low[1], env.action_space.high[1], num=action_res[1])
w = np.linspace(env.action_space.low[2], env.action_space.high[2], num=action_res[2])
for i in range(action_res[0]):
    for j in range(action_res[1]):
        for k in range(action_res[2]):
            s = action_res[2] * action_res[1] * i + action_res[2] * j + k
            action_map[s, :] = [u[i], v[j], w[k]]
'''

[2018-06-05 23:14:35,989] Making new env: Reacher-v1


'\naction_res = [6, 6, 6]\nN_A = 6*6*6\naction_map = np.zeros([np.prod(action_res), 3])\nu = np.linspace(env.action_space.low[0], env.action_space.high[0], num=action_res[0])\nv = np.linspace(env.action_space.low[1], env.action_space.high[1], num=action_res[1])\nw = np.linspace(env.action_space.low[2], env.action_space.high[2], num=action_res[2])\nfor i in range(action_res[0]):\n    for j in range(action_res[1]):\n        for k in range(action_res[2]):\n            s = action_res[2] * action_res[1] * i + action_res[2] * j + k\n            action_map[s, :] = [u[i], v[j], w[k]]\n'

In [2]:
class ExpReplay(object):
    def __init__(self, memory_size, state_dim, act_dim, batch_size):
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0
    def fifo(self, state, action, reward, next_state):
        self.states[self.current] = state
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.next_states[self.current] = next_state
        self.current = (self.current + 1) % self.memory_size
        self.count = self.count + 1
    def add_trajectory(self, states, actions, rewards, next_states):
        num = len(observes)
        for i in range(0, num):
            self.fifo(states[i], actions[i], rewards[i], next_states[i])
    def sampling(self):
        indexes = np.random.randint(min(self.count, self.memory_size), size=self.batch_size)
        states = self.states[indexes]
        actions = self.actions[indexes]
        rewards = self.rewards[indexes]
        next_states = self.next_states[indexes]
        states = states.reshape((-1, self.state_dim))
        actions = actions.reshape((-1, 1))
        rewards = rewards.reshape((-1, 1))
        next_states = next_states.reshape((-1, self.state_dim))
        return states, actions, rewards, next_states

In [3]:
class Memory(object):
    def __init__(self, memory_size, state_dim, act_dim):
        self.memory_size = memory_size
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0
    def fifo(self, state, action, reward, next_state):
        self.states[self.current] = state
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.next_states[self.current] = next_state
        self.current = (self.current + 1) % self.memory_size
        self.count = self.count + 1
    def add_trajectory(self, states, actions, rewards, next_states):
        num = len(observes)
        for i in range(0, num):
            self.fifo(states[i], actions[i], rewards[i], next_states[i])
    def sampling(self):
        randpermlist = np.random.permutation(self.count)
        states = self.states[randpermlist]
        actions = self.actions[randpermlist]
        rewards = self.rewards[randpermlist]
        next_states = self.next_states[randpermlist]
        states = states.reshape((-1, self.state_dim))
        actions = actions.reshape((-1, 1))
        rewards = rewards.reshape((-1, 1))
        next_states = next_states.reshape((-1, self.state_dim))
        return states, actions, rewards, next_states
    def empty_memory(self):
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0

In [4]:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, lr=0.0005):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, (None, n_features), "state")
        self.a = tf.placeholder(tf.float32, (None, n_actions), "act")
        self.td_error = tf.placeholder(tf.float32, (None, 1), "td_error")  # TD_error]
        self.eps = 1
        self.eps_decay_rate = 0.95
        self.eps_min = 0

        with tf.variable_scope('Actor'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=32,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            l2 = tf.layers.dense(
                inputs=l1,
                units=16,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l2'
            )
            
            self.acts_prob = tf.layers.dense(
                inputs=l2,
                units=n_actions,    # output units
                activation=tf.contrib.sparsemax.sparsemax,   # get action probabilities
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='acts_prob'
            )
        '''    
        with tf.variable_scope('exp_v'):
            log_prob = tf.log(tf.reduce_sum(
                tf.multiply(self.acts_prob, self.a), 1, keep_dims = True))
            self.loss = tf.reduce_mean(log_prob * self.td_error)  # advantage (TD_error) guided loss

        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(-self.loss)
        '''
        with tf.variable_scope('exp_v'):
            self.error = self.td_error + ALPHA*0.5*tf.reduce_sum(
                tf.multiply(self.acts_prob, self.acts_prob), 1, keep_dims=True) + ALPHA*0.5 - ALPHA*tf.reduce_sum(
                tf.multiply(self.acts_prob, self.a), 1, keep_dims = True)
            # TD error + ALPHA(0.5*pi^2 + 0.5 - pi)
            self.loss = tf.reduce_sum(tf.square(self.error))

        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)  
            
    def learn(self, s, a, td):
        BATCH_SIZE = 200
        NITER = int(s.shape[0]/BATCH_SIZE)
        for i in range(0, NITER):
            feed_dict = {self.s: s[i*BATCH_SIZE:(i+1)*BATCH_SIZE], 
                         self.a: a[i*BATCH_SIZE:(i+1)*BATCH_SIZE], 
                         self.td_error: td[i*BATCH_SIZE:(i+1)*BATCH_SIZE]}
            _, loss = self.sess.run([self.train_op, self.loss], feed_dict)
        return 0

    def choose_action(self, s):
        s = s[np.newaxis, :]
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        probs = probs.ravel()
        probs = np.ones(probs.shape[0]) * self.eps / probs.shape[0] + probs * (1. - self.eps)
        probs = probs/np.sum(probs)
        return np.random.choice(np.arange(probs.shape[0]), p=probs)   # return a int
    
    def update_policy(self):
        self.eps = np.max((self.eps*self.eps_decay_rate, self.eps_min))
        
    def print_prob(self, s):
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        probs = probs.ravel()
        probs = np.ones(probs.shape[0]) * self.eps / probs.shape[0] + probs * (1. - self.eps)
        probs = probs/np.sum(probs)
        print(probs)
        return probs

In [5]:
class Critic(object):
    def __init__(self, sess, n_features, lr=0.01):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, (None, n_features), "state")
        self.v_ = tf.placeholder(tf.float32, (None, 1), "v_next")
        self.r = tf.placeholder(tf.float32, (None, 1), 'r')

        with tf.variable_scope('Critic'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=32,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )
            
            l2 = tf.layers.dense(
                inputs=l1,
                units=16,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l2'
            )

            self.v = tf.layers.dense(
                inputs=l2,
                units=1,  # output units
                activation=None,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='V'
            )
            
        with tf.variable_scope('squared_TD_error'):
            self.td_error = self.r + GAMMA * self.v_ - self.v
            self.loss = tf.reduce_sum(tf.square(self.td_error))    # TD_error = (r+gamma*V_next) - V_eval
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)

    def learn(self, s, r, s_):
        BATCH_SIZE = 200
        NITER = int(s.shape[0]/BATCH_SIZE)
        for i in range(0, NITER):
            v_ = self.sess.run(self.v, {self.s: s_[i*BATCH_SIZE:(i+1)*BATCH_SIZE]})
            td_error, _ = self.sess.run([self.td_error, self.train_op],
                                              {self.s: s[i*BATCH_SIZE:(i+1)*BATCH_SIZE], self.v_: v_, 
                                               self.r: r[i*BATCH_SIZE:(i+1)*BATCH_SIZE]})
        return 0
    
    def get_error(self, s, r, s_):
        v_ = self.sess.run(self.v, {self.s: s_})
        td_error = self.sess.run(self.td_error,
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error
        

In [6]:
sess = tf.Session()

actor = Actor(sess, n_features=N_F, n_actions=N_A, lr=LR_A)
# critic = Critic(sess, n_features=N_F, lr=LR_C)
critic = Critic(sess, n_features=N_F, lr=LR_C)     # we need a good teacher, so the teacher should learn faster than the actor
MEMORY_SIZE = 10000
BATCH_SIZE = 1000
memory = Memory(MEMORY_SIZE, N_F, N_A)
# replay = ExpReplay(MEMORY_SIZE, N_F, N_A, BATCH_SIZE)
sess.run(tf.global_variables_initializer())

if OUTPUT_GRAPH:
    tf.summary.FileWriter("logs/", sess.graph)

for i_episode in range(MAX_EPISODE):
    s = env.reset()
    t = 0
    track_r = []
    while True:
        if RENDER: env.render()

        a = actor.choose_action(s)
        s_, r, done, info = env.step(action_map[a, :])

        track_r.append(2*info['reward_dist']+info['reward_ctrl'])
       
        #td_error = critic.learn(s, r, s_)  # gradient = grad[r + gamma * V(s_) - V(s)]
        #actor.learn(s, a, td_error)     # true_gradient = grad[logPi(s,a) * td_error]
        memory.fifo(s, a, r, s_)
        s = s_
        t += 1
        if t >= MAX_EP_STEPS: #done or
            ep_rs_sum = sum(track_r)

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.95 + ep_rs_sum * 0.05
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            break
        
    if (i_episode + 1)%10 == 0:
        states, actions, rewards, next_states = memory.sampling()
        mx_actions = np.zeros((actions.shape[0], N_A))
        for i in range(0, states.shape[0]):
            mx_actions[i, int(actions[i, 0])] = 1
        td_errors = critic.get_error(states, rewards, next_states)
        actor.learn(states, mx_actions, td_errors)
        critic.learn(states, rewards, next_states)
        actor.update_policy()
        memory.empty_memory()

('episode:', 0, '  reward:', -112)
('episode:', 1, '  reward:', -113)
('episode:', 2, '  reward:', -113)
('episode:', 3, '  reward:', -113)
('episode:', 4, '  reward:', -114)
('episode:', 5, '  reward:', -115)
('episode:', 6, '  reward:', -114)
('episode:', 7, '  reward:', -115)
('episode:', 8, '  reward:', -115)
('episode:', 9, '  reward:', -117)
('episode:', 10, '  reward:', -117)
('episode:', 11, '  reward:', -116)
('episode:', 12, '  reward:', -116)
('episode:', 13, '  reward:', -116)
('episode:', 14, '  reward:', -116)
('episode:', 15, '  reward:', -117)
('episode:', 16, '  reward:', -117)
('episode:', 17, '  reward:', -118)
('episode:', 18, '  reward:', -118)
('episode:', 19, '  reward:', -117)
('episode:', 20, '  reward:', -119)
('episode:', 21, '  reward:', -119)
('episode:', 22, '  reward:', -121)
('episode:', 23, '  reward:', -122)
('episode:', 24, '  reward:', -122)
('episode:', 25, '  reward:', -123)
('episode:', 26, '  reward:', -122)
('episode:', 27, '  reward:', -121)
('

('episode:', 227, '  reward:', -107)
('episode:', 228, '  reward:', -108)
('episode:', 229, '  reward:', -108)
('episode:', 230, '  reward:', -109)
('episode:', 231, '  reward:', -109)
('episode:', 232, '  reward:', -108)
('episode:', 233, '  reward:', -107)
('episode:', 234, '  reward:', -107)
('episode:', 235, '  reward:', -106)
('episode:', 236, '  reward:', -105)
('episode:', 237, '  reward:', -105)
('episode:', 238, '  reward:', -106)
('episode:', 239, '  reward:', -106)
('episode:', 240, '  reward:', -105)
('episode:', 241, '  reward:', -106)
('episode:', 242, '  reward:', -104)
('episode:', 243, '  reward:', -104)
('episode:', 244, '  reward:', -105)
('episode:', 245, '  reward:', -105)
('episode:', 246, '  reward:', -104)
('episode:', 247, '  reward:', -104)
('episode:', 248, '  reward:', -105)
('episode:', 249, '  reward:', -105)
('episode:', 250, '  reward:', -105)
('episode:', 251, '  reward:', -105)
('episode:', 252, '  reward:', -105)
('episode:', 253, '  reward:', -104)
(

('episode:', 453, '  reward:', -85)
('episode:', 454, '  reward:', -86)
('episode:', 455, '  reward:', -86)
('episode:', 456, '  reward:', -86)
('episode:', 457, '  reward:', -86)
('episode:', 458, '  reward:', -86)
('episode:', 459, '  reward:', -86)
('episode:', 460, '  reward:', -86)
('episode:', 461, '  reward:', -86)
('episode:', 462, '  reward:', -86)
('episode:', 463, '  reward:', -86)
('episode:', 464, '  reward:', -87)
('episode:', 465, '  reward:', -86)
('episode:', 466, '  reward:', -86)
('episode:', 467, '  reward:', -86)
('episode:', 468, '  reward:', -86)
('episode:', 469, '  reward:', -86)
('episode:', 470, '  reward:', -86)
('episode:', 471, '  reward:', -86)
('episode:', 472, '  reward:', -85)
('episode:', 473, '  reward:', -85)
('episode:', 474, '  reward:', -85)
('episode:', 475, '  reward:', -84)
('episode:', 476, '  reward:', -84)
('episode:', 477, '  reward:', -84)
('episode:', 478, '  reward:', -85)
('episode:', 479, '  reward:', -85)
('episode:', 480, '  reward:

('episode:', 681, '  reward:', -74)
('episode:', 682, '  reward:', -75)
('episode:', 683, '  reward:', -74)
('episode:', 684, '  reward:', -75)
('episode:', 685, '  reward:', -75)
('episode:', 686, '  reward:', -76)
('episode:', 687, '  reward:', -76)
('episode:', 688, '  reward:', -76)
('episode:', 689, '  reward:', -77)
('episode:', 690, '  reward:', -78)
('episode:', 691, '  reward:', -78)
('episode:', 692, '  reward:', -78)
('episode:', 693, '  reward:', -77)
('episode:', 694, '  reward:', -77)
('episode:', 695, '  reward:', -76)
('episode:', 696, '  reward:', -76)
('episode:', 697, '  reward:', -76)
('episode:', 698, '  reward:', -76)
('episode:', 699, '  reward:', -77)
('episode:', 700, '  reward:', -76)
('episode:', 701, '  reward:', -77)
('episode:', 702, '  reward:', -77)
('episode:', 703, '  reward:', -78)
('episode:', 704, '  reward:', -77)
('episode:', 705, '  reward:', -76)
('episode:', 706, '  reward:', -76)
('episode:', 707, '  reward:', -76)
('episode:', 708, '  reward:

('episode:', 909, '  reward:', -69)
('episode:', 910, '  reward:', -68)
('episode:', 911, '  reward:', -68)
('episode:', 912, '  reward:', -69)
('episode:', 913, '  reward:', -68)
('episode:', 914, '  reward:', -68)
('episode:', 915, '  reward:', -67)
('episode:', 916, '  reward:', -67)
('episode:', 917, '  reward:', -68)
('episode:', 918, '  reward:', -67)
('episode:', 919, '  reward:', -66)
('episode:', 920, '  reward:', -66)
('episode:', 921, '  reward:', -68)
('episode:', 922, '  reward:', -67)
('episode:', 923, '  reward:', -67)
('episode:', 924, '  reward:', -66)
('episode:', 925, '  reward:', -66)
('episode:', 926, '  reward:', -66)
('episode:', 927, '  reward:', -66)
('episode:', 928, '  reward:', -66)
('episode:', 929, '  reward:', -65)
('episode:', 930, '  reward:', -64)
('episode:', 931, '  reward:', -64)
('episode:', 932, '  reward:', -66)
('episode:', 933, '  reward:', -66)
('episode:', 934, '  reward:', -67)
('episode:', 935, '  reward:', -67)
('episode:', 936, '  reward:

...

('episode:', 49023, '  reward:', -43)
('episode:', 49024, '  reward:', -44)
('episode:', 49025, '  reward:', -44)
('episode:', 49026, '  reward:', -44)
('episode:', 49027, '  reward:', -45)
('episode:', 49028, '  reward:', -44)
('episode:', 49029, '  reward:', -44)
('episode:', 49030, '  reward:', -45)
('episode:', 49031, '  reward:', -44)
('episode:', 49032, '  reward:', -43)
('episode:', 49033, '  reward:', -43)
('episode:', 49034, '  reward:', -43)
('episode:', 49035, '  reward:', -42)
('episode:', 49036, '  reward:', -42)
('episode:', 49037, '  reward:', -42)
('episode:', 49038, '  reward:', -41)
('episode:', 49039, '  reward:', -41)
('episode:', 49040, '  reward:', -41)
('episode:', 49041, '  reward:', -41)
('episode:', 49042, '  reward:', -40)
('episode:', 49043, '  reward:', -40)
('episode:', 49044, '  reward:', -40)
('episode:', 49045, '  reward:', -41)
('episode:', 49046, '  reward:', -40)
('episode:', 49047, '  reward:', -40)
('episode:', 49048, '  reward:', -39)
('episode:',

('episode:', 49240, '  reward:', -47)
('episode:', 49241, '  reward:', -46)
('episode:', 49242, '  reward:', -46)
('episode:', 49243, '  reward:', -46)
('episode:', 49244, '  reward:', -45)
('episode:', 49245, '  reward:', -44)
('episode:', 49246, '  reward:', -44)
('episode:', 49247, '  reward:', -45)
('episode:', 49248, '  reward:', -44)
('episode:', 49249, '  reward:', -43)
('episode:', 49250, '  reward:', -44)
('episode:', 49251, '  reward:', -43)
('episode:', 49252, '  reward:', -42)
('episode:', 49253, '  reward:', -43)
('episode:', 49254, '  reward:', -43)
('episode:', 49255, '  reward:', -43)
('episode:', 49256, '  reward:', -43)
('episode:', 49257, '  reward:', -44)
('episode:', 49258, '  reward:', -44)
('episode:', 49259, '  reward:', -45)
('episode:', 49260, '  reward:', -45)
('episode:', 49261, '  reward:', -44)
('episode:', 49262, '  reward:', -44)
('episode:', 49263, '  reward:', -45)
('episode:', 49264, '  reward:', -46)
('episode:', 49265, '  reward:', -46)
('episode:',

('episode:', 49457, '  reward:', -45)
('episode:', 49458, '  reward:', -44)
('episode:', 49459, '  reward:', -45)
('episode:', 49460, '  reward:', -46)
('episode:', 49461, '  reward:', -45)
('episode:', 49462, '  reward:', -46)
('episode:', 49463, '  reward:', -46)
('episode:', 49464, '  reward:', -46)
('episode:', 49465, '  reward:', -46)
('episode:', 49466, '  reward:', -46)
('episode:', 49467, '  reward:', -45)
('episode:', 49468, '  reward:', -46)
('episode:', 49469, '  reward:', -46)
('episode:', 49470, '  reward:', -46)
('episode:', 49471, '  reward:', -46)
('episode:', 49472, '  reward:', -47)
('episode:', 49473, '  reward:', -46)
('episode:', 49474, '  reward:', -46)
('episode:', 49475, '  reward:', -46)
('episode:', 49476, '  reward:', -46)
('episode:', 49477, '  reward:', -47)
('episode:', 49478, '  reward:', -48)
('episode:', 49479, '  reward:', -48)
('episode:', 49480, '  reward:', -47)
('episode:', 49481, '  reward:', -47)
('episode:', 49482, '  reward:', -46)
('episode:',

('episode:', 49673, '  reward:', -47)
('episode:', 49674, '  reward:', -47)
('episode:', 49675, '  reward:', -46)
('episode:', 49676, '  reward:', -46)
('episode:', 49677, '  reward:', -46)
('episode:', 49678, '  reward:', -46)
('episode:', 49679, '  reward:', -46)
('episode:', 49680, '  reward:', -45)
('episode:', 49681, '  reward:', -45)
('episode:', 49682, '  reward:', -44)
('episode:', 49683, '  reward:', -43)
('episode:', 49684, '  reward:', -43)
('episode:', 49685, '  reward:', -44)
('episode:', 49686, '  reward:', -44)
('episode:', 49687, '  reward:', -45)
('episode:', 49688, '  reward:', -45)
('episode:', 49689, '  reward:', -45)
('episode:', 49690, '  reward:', -46)
('episode:', 49691, '  reward:', -46)
('episode:', 49692, '  reward:', -46)
('episode:', 49693, '  reward:', -46)
('episode:', 49694, '  reward:', -46)
('episode:', 49695, '  reward:', -46)
('episode:', 49696, '  reward:', -46)
('episode:', 49697, '  reward:', -47)
('episode:', 49698, '  reward:', -46)
('episode:',

('episode:', 49890, '  reward:', -44)
('episode:', 49891, '  reward:', -45)
('episode:', 49892, '  reward:', -44)
('episode:', 49893, '  reward:', -45)
('episode:', 49894, '  reward:', -44)
('episode:', 49895, '  reward:', -43)
('episode:', 49896, '  reward:', -43)
('episode:', 49897, '  reward:', -43)
('episode:', 49898, '  reward:', -42)
('episode:', 49899, '  reward:', -43)
('episode:', 49900, '  reward:', -43)
('episode:', 49901, '  reward:', -43)
('episode:', 49902, '  reward:', -43)
('episode:', 49903, '  reward:', -43)
('episode:', 49904, '  reward:', -43)
('episode:', 49905, '  reward:', -44)
('episode:', 49906, '  reward:', -45)
('episode:', 49907, '  reward:', -44)
('episode:', 49908, '  reward:', -44)
('episode:', 49909, '  reward:', -44)
('episode:', 49910, '  reward:', -44)
('episode:', 49911, '  reward:', -44)
('episode:', 49912, '  reward:', -43)
('episode:', 49913, '  reward:', -43)
('episode:', 49914, '  reward:', -43)
('episode:', 49915, '  reward:', -43)
('episode:',

In [7]:
for i_episode in range(0, 200):
    s = env.reset()
    t = 0
    track_r = []
    while True:
        env.render()

        a = actor.choose_action(s)
        s_, r, done, info = env.step(action_map[a, :])
        s = s_
        t += 1
        if t >= MAX_EP_STEPS: #done or
            ep_rs_sum = sum(track_r)

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.95 + ep_rs_sum * 0.05
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            break

('episode:', 0, '  reward:', -43)
('episode:', 1, '  reward:', -41)
('episode:', 2, '  reward:', -39)
('episode:', 3, '  reward:', -37)
('episode:', 4, '  reward:', -35)
('episode:', 5, '  reward:', -33)
('episode:', 6, '  reward:', -32)
('episode:', 7, '  reward:', -30)
('episode:', 8, '  reward:', -29)
('episode:', 9, '  reward:', -27)
('episode:', 10, '  reward:', -26)
('episode:', 11, '  reward:', -24)
('episode:', 12, '  reward:', -23)
('episode:', 13, '  reward:', -22)
('episode:', 14, '  reward:', -21)
('episode:', 15, '  reward:', -20)
('episode:', 16, '  reward:', -19)
('episode:', 17, '  reward:', -18)
('episode:', 18, '  reward:', -17)
('episode:', 19, '  reward:', -16)
('episode:', 20, '  reward:', -15)
('episode:', 21, '  reward:', -14)
('episode:', 22, '  reward:', -14)
('episode:', 23, '  reward:', -13)
('episode:', 24, '  reward:', -12)
('episode:', 25, '  reward:', -12)
('episode:', 26, '  reward:', -11)
('episode:', 27, '  reward:', -10)
('episode:', 28, '  reward:', 